# ($t = \pi/6$) Expected state after each step in DME using parameterized SWAP gate

In [1]:
import numpy as np
import scipy

t = np.pi/6
t_ZZ = t/2


Define partial trace and Hermitian conjugate function

In [2]:

def ptrace(matrix,index=1,n1=2,n2=2):
    # Partial trace function for density matrices.
    current_tensor=matrix.reshape([n1, n2, n1, n2])
    if index == 1:
        return np.trace(current_tensor, axis1=1, axis2=3)
    elif index == 2:
        return np.trace(current_tensor, axis1=0, axis2=2)
    else:
        print('Error')

def dagger(u):
    return np.asarray(np.matrix(u).H)

Pauli matrices etc

In [3]:
X = np.array([[0,1],
              [1,0]])

Y = np.array([[0,-1j],
              [1j,0]])

Z = np.array([[1,0],
              [0,-1]])

identity = np.array([[1,0],
              [0,1]])

swap = np.array([[1, 0, 0, 0],
            [0, 0, 1, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1]])

XX = np.kron(X,X)
YY = np.kron(Y,Y)
ZZ = np.kron(Z,Z)


Define the qubit initial density matrices

In [4]:

ket_plus = np.array([[1/np.sqrt(2)],[1/np.sqrt(2)]])
rho_plus = np.outer(ket_plus,ket_plus)

ket_0 = np.array([[1],[0]])
rho_0 = np.outer(ket_0,ket_0)

Define the unitaries

In [5]:
# Parameterized SWAP
U_ZZ = scipy.linalg.expm(-1j*t_ZZ*ZZ)
U_YY = scipy.linalg.expm(-1j*t_ZZ*YY)
U_XX = scipy.linalg.expm(-1j*t_ZZ*XX)

U_XXZ = U_XX @ U_YY @ U_ZZ * np.exp(-1j * t_ZZ)
np.round(U_XXZ @ np.kron(rho_0,rho_plus) @ dagger(U_XXZ),3)

array([[ 0.5  +0.j   ,  0.375-0.217j,  0.125+0.217j,  0.   +0.j   ],
       [ 0.375+0.217j,  0.375+0.j   , -0.   +0.217j,  0.   +0.j   ],
       [ 0.125-0.217j, -0.   -0.217j,  0.125+0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ]])

In [6]:
# Native SWAP
U = scipy.linalg.expm(-1j*t*swap)
np.round(U @ np.kron(rho_0,rho_plus) @ dagger (U),3)

array([[0.5  -0.j   , 0.375-0.217j, 0.125+0.217j, 0.   +0.j   ],
       [0.375+0.217j, 0.375+0.j   , 0.   +0.217j, 0.   +0.j   ],
       [0.125-0.217j, 0.   -0.217j, 0.125+0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ]])

In [7]:
# Compare parameterized and native
np.linalg.norm(U_XXZ-U)

1.9547948663258415e-16

In [8]:
print("Unitary for 1 step of dSWAP interaction:")
print(np.round(U,3))

Unitary for 1 step of dSWAP interaction:
[[0.866-0.5j 0.   +0.j  0.   +0.j  0.   +0.j ]
 [0.   +0.j  0.866+0.j  0.   -0.5j 0.   +0.j ]
 [0.   +0.j  0.   -0.5j 0.866+0.j  0.   +0.j ]
 [0.   +0.j  0.   +0.j  0.   +0.j  0.866-0.5j]]


## Iteration A

In [9]:
# Parameterized SWAP
rho_A = np.kron(rho_0,rho_plus)
rho_A1 = U_ZZ @ rho_A @ dagger(U_ZZ)
rho_A2 = U_YY @ rho_A1 @ dagger(U_YY)
rho_A3 = U_XX @ rho_A2 @ dagger(U_XX)
# Native SWAP
rho_A3_swap = U @ rho_A @ dagger(U)

In [10]:
print('A initial q0 qubit state:')
print(np.round(ptrace(rho_A),3))
print('A1 (after Z0Z1) q0 qubit state:')
print(np.round(ptrace(rho_A1),3))
print('A2 (after Y0Y1) q0 qubit state:')
print(np.round(ptrace(rho_A2),3))
print('A3 (after X0X1) q0 qubit state:')
print(np.round(ptrace(rho_A3),3))
print('---')
print('A3 (using native swap) q0 qubit state:')
print(np.round(ptrace(rho_A3_swap),3))

A initial q0 qubit state:
[[1. 0.]
 [0. 0.]]
A1 (after Z0Z1) q0 qubit state:
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
A2 (after Y0Y1) q0 qubit state:
[[0.933+0.j 0.125+0.j]
 [0.125+0.j 0.067+0.j]]
A3 (after X0X1) q0 qubit state:
[[0.875-0.j    0.125+0.217j]
 [0.125-0.217j 0.125+0.j   ]]
---
A3 (using native swap) q0 qubit state:
[[0.875-0.j    0.125+0.217j]
 [0.125-0.217j 0.125+0.j   ]]


In [11]:
print('A q0-q1 initial state:')
print(np.round(rho_A,3))
print('A3 q0-q1 2 qubit state using XXZ')
print(np.round(rho_A3,3))
print('A3 q0-q1 2 qubit state using swap')
print(np.round(rho_A3_swap,3))

A q0-q1 initial state:
[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
A3 q0-q1 2 qubit state using XXZ
[[ 0.5  -0.j     0.375-0.217j  0.125+0.217j  0.   +0.j   ]
 [ 0.375+0.217j  0.375+0.j    -0.   +0.217j  0.   +0.j   ]
 [ 0.125-0.217j  0.   -0.217j  0.125+0.j    -0.   +0.j   ]
 [-0.   +0.j     0.   -0.j    -0.   +0.j     0.   +0.j   ]]
A3 q0-q1 2 qubit state using swap
[[0.5  -0.j    0.375-0.217j 0.125+0.217j 0.   +0.j   ]
 [0.375+0.217j 0.375+0.j    0.   +0.217j 0.   +0.j   ]
 [0.125-0.217j 0.   -0.217j 0.125+0.j    0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   ]]


In [12]:
# Compare parameterized and native
np.linalg.norm(rho_A3_swap-rho_A3)

1.724283721331207e-16

## Iteration B

In [13]:
# Parameterized SWAP
rho_B = np.kron(ptrace(rho_A3),rho_plus)
rho_B1 = U_ZZ @ rho_B @ dagger(U_ZZ)
rho_B2 = U_YY @ rho_B1 @ dagger(U_YY)
rho_B3 = U_XX @ rho_B2 @ dagger(U_XX)
# Native SWAP
rho_B3_swap = U @ rho_B @ dagger(U)

In [14]:
print('B initial q0 qubit state:')
print(np.round(ptrace(rho_B),3))
print('B1 (after Z0Z2) q0 qubit state:')
print(np.round(ptrace(rho_B1),3))
print('B2 (after Y0Y2) q0 qubit state:')
print(np.round(ptrace(rho_B2),3))
print('B3 (after X0X2) q0 qubit state:')
print(np.round(ptrace(rho_B3),3))
print('---')
print('B3 (using native swap) q0 qubit state:')
print(np.round(ptrace(rho_B3_swap),3))

B initial q0 qubit state:
[[0.875-0.j    0.125+0.217j]
 [0.125-0.217j 0.125+0.j   ]]
B1 (after Z0Z2) q0 qubit state:
[[0.875+0.j    0.108+0.187j]
 [0.108-0.187j 0.125+0.j   ]]
B2 (after Y0Y2) q0 qubit state:
[[0.825+0.j    0.219+0.187j]
 [0.219-0.187j 0.175-0.j   ]]
B3 (after X0X2) q0 qubit state:
[[0.688-0.j    0.219+0.325j]
 [0.219-0.325j 0.312+0.j   ]]
---
B3 (using native swap) q0 qubit state:
[[0.687+0.j    0.219+0.325j]
 [0.219-0.325j 0.312-0.j   ]]


In [15]:
print('B q0-q2 initial state:')
print(np.round(rho_B,3))
print('B3 q0-q2 2 qubit state using XXZ')
print(np.round(rho_B3,3))
print('B3 q0-q2 2 qubit state using swap')
print(np.round(rho_B3_swap,3))

B q0-q2 initial state:
[[0.437-0.j    0.437-0.j    0.062+0.108j 0.062+0.108j]
 [0.437-0.j    0.437-0.j    0.062+0.108j 0.062+0.108j]
 [0.062-0.108j 0.062-0.108j 0.062+0.j    0.062+0.j   ]
 [0.062-0.108j 0.062-0.108j 0.062+0.j    0.062+0.j   ]]
B3 q0-q2 2 qubit state using XXZ
[[0.437-0.j    0.297-0.135j 0.203+0.244j 0.062+0.108j]
 [0.297+0.135j 0.25 +0.j    0.062+0.217j 0.016+0.081j]
 [0.203-0.244j 0.062-0.217j 0.25 +0.j    0.109+0.027j]
 [0.062-0.108j 0.016-0.081j 0.109-0.027j 0.062+0.j   ]]
B3 q0-q2 2 qubit state using swap
[[0.437+0.j    0.297-0.135j 0.203+0.244j 0.062+0.108j]
 [0.297+0.135j 0.25 +0.j    0.062+0.217j 0.016+0.081j]
 [0.203-0.244j 0.062-0.217j 0.25 +0.j    0.109+0.027j]
 [0.062-0.108j 0.016-0.081j 0.109-0.027j 0.062-0.j   ]]


In [16]:
# Compare parameterized and native
np.linalg.norm(rho_B3_swap-rho_B3)

1.6389235550545282e-16

## Iteration C

In [17]:
# Parameterized SWAP
rho_C = np.kron(ptrace(rho_B3),rho_plus)
rho_C1 = U_ZZ @ rho_C @ dagger(U_ZZ)
rho_C2 = U_YY @ rho_C1 @ dagger(U_YY)
rho_C3 = U_XX @ rho_C2 @ dagger(U_XX)

# Native SWAP
rho_C3_swap = U @ rho_C @ dagger(U)

In [18]:
print('C initial q0 qubit state:')
print(np.round(ptrace(rho_C),3))
print('C1 (after Z0Z3) q0 qubit state:')
print(np.round(ptrace(rho_C1),3))
print('C2 (after Y0Y3) q0 qubit state:')
print(np.round(ptrace(rho_C2),3))
print('C3 (after X0X3) q0 qubit state:')
print(np.round(ptrace(rho_C3),3))
print('---')
print('C3 (using native swap) q0 qubit state:')
print(np.round(ptrace(rho_C3_swap),3))

C initial q0 qubit state:
[[0.687-0.j    0.219+0.325j]
 [0.219-0.325j 0.312+0.j   ]]
C1 (after Z0Z3) q0 qubit state:
[[0.687+0.j    0.189+0.281j]
 [0.189-0.281j 0.312+0.j   ]]
C2 (after Y0Y3) q0 qubit state:
[[0.662-0.j    0.289+0.281j]
 [0.289-0.281j 0.338+0.j   ]]
C3 (after X0X3) q0 qubit state:
[[0.5  +0.j    0.289+0.325j]
 [0.289-0.325j 0.5  +0.j   ]]
---
C3 (using native swap) q0 qubit state:
[[0.5  +0.j    0.289+0.325j]
 [0.289-0.325j 0.5  -0.j   ]]


In [19]:
print('C q0-q3 initial state:')
print(np.round(rho_C,3))
print('C3 q0-q3 2 qubit state using XXZ')
print(np.round(rho_C3,3))
print('C3 q0-q3 2 qubit state using swap')
print(np.round(rho_C3_swap,3))

C q0-q3 initial state:
[[0.344-0.j    0.344-0.j    0.109+0.162j 0.109+0.162j]
 [0.344-0.j    0.344-0.j    0.109+0.162j 0.109+0.162j]
 [0.109-0.162j 0.109-0.162j 0.156+0.j    0.156+0.j   ]
 [0.109-0.162j 0.109-0.162j 0.156+0.j    0.156+0.j   ]]
C3 q0-q3 2 qubit state using XXZ
[[0.344-0.j    0.215-0.061j 0.238+0.223j 0.109+0.162j]
 [0.215+0.061j 0.156+0.j    0.109+0.162j 0.051+0.101j]
 [0.238-0.223j 0.109-0.162j 0.344+0.j    0.215+0.061j]
 [0.109-0.162j 0.051-0.101j 0.215-0.061j 0.156-0.j   ]]
C3 q0-q3 2 qubit state using swap
[[0.344+0.j    0.215-0.061j 0.238+0.223j 0.109+0.162j]
 [0.215+0.061j 0.156+0.j    0.109+0.162j 0.051+0.101j]
 [0.238-0.223j 0.109-0.162j 0.344-0.j    0.215+0.061j]
 [0.109-0.162j 0.051-0.101j 0.215-0.061j 0.156+0.j   ]]


In [20]:
# Compare parameterized and native
np.linalg.norm(rho_C3_swap-rho_C3)

1.9675159943996247e-16

## Output

In [21]:
single_qubit_dm_pi_6 = {
    '1Q_dm_A1': ptrace(rho_A1),
    '1Q_dm_A2': ptrace(rho_A2),
    '1Q_dm_A3': ptrace(rho_A3),
    '1Q_dm_B1': ptrace(rho_B1),
    '1Q_dm_B2': ptrace(rho_B2),
    '1Q_dm_B3': ptrace(rho_B3),
    '1Q_dm_C1': ptrace(rho_C1),
    '1Q_dm_C2': ptrace(rho_C2),
    '1Q_dm_C3': ptrace(rho_C3),
}



In [22]:
two_qubit_dm_pi_6 = {
    '2Q_dm_A3': rho_A3,
    '2Q_dm_B3': rho_B3,
    '2Q_dm_C3': rho_C3,
}

In [23]:
import csv

def write_to_csv(filename, data):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Key', 'Values'])
        for key, values in data.items():
            formatted_values = [[str(elem) for elem in row] for row in values]
            writer.writerow([key, formatted_values])

write_to_csv('t_pi_6_DME_1q.csv', single_qubit_dm_pi_6)
write_to_csv('t_pi_6_DME_2q.csv', two_qubit_dm_pi_6)


In [24]:
import csv
import numpy as np
import ast  # Safer than eval for parsing strings to lists

def load_from_csv(filename):
    data_loaded = {}
    with open(filename, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            key = row[0]
            # Safely convert the string representation of the array back to a list
            values = ast.literal_eval(row[1])  
            data_loaded[key] = np.asarray(values, dtype=np.complex128)  # Use complex128 for complex numbers
    return data_loaded

single_qubit_dm_pi_6_loaded = load_from_csv('t_pi_6_DME_1q.csv')
two_qubit_dm_pi_6_loaded = load_from_csv('t_pi_6_DME_2q.csv')

# Check the loaded dictionaries
print(single_qubit_dm_pi_6_loaded['1Q_dm_A1'])
print(two_qubit_dm_pi_6_loaded['2Q_dm_B3'])


[['(0.9999999999999998+0j)' '0j']
 ['0j' '0j']]
[['(0.4374999999999999-1.734723475976807e-17j)'
  '(0.29687500000000006-0.13531646934131855j)'
  '(0.2031249999999999+0.24356964481437332j)'
  '(0.062499999999999944+0.10825317547305478j)']
 ['(0.296875+0.13531646934131852j)'
  '(0.25000000000000006+1.5612511283791264e-17j)'
  '(0.06249999999999995+0.21650635094610965j)'
  '(0.015624999999999986+0.08118988160479111j)']
 ['(0.2031249999999999-0.2435696448143733j)'
  '(0.062499999999999965-0.21650635094610962j)' '(0.2499999999999998+0j)'
  '(0.10937499999999989+0.02706329386826368j)']
 ['(0.06249999999999997-0.10825317547305478j)'
  '(0.015624999999999981-0.08118988160479113j)'
  '(0.10937499999999992-0.027063293868263675j)'
  '(0.06249999999999994+1.734723475976807e-18j)']]


## Bloch sphere

In [25]:
from qutip import Bloch, Qobj
import qutip
from moviepy.editor import ImageSequenceClip

states = [Qobj(ptrace(rho_A1)),
          Qobj(ptrace(rho_A2)),
          Qobj(ptrace(rho_A3)),
          Qobj(ptrace(rho_B1)),
          Qobj(ptrace(rho_B2)),
          Qobj(ptrace(rho_B3)),
          Qobj(ptrace(rho_C1)),
          Qobj(ptrace(rho_C2)),
          Qobj(ptrace(rho_C3))]


b = qutip.Bloch()
b.vector_color = ['r']
b.view = [-40, 30]

for i, state in enumerate(states):
    b.clear()
    b.add_states(state)
    b.save(dirc='temp')  # Custom filename for each state


In [26]:

# Create and save a video clip
clip = ImageSequenceClip([f'temp/bloch_{i}.png' for i in range(9)], fps=5)
clip.write_gif("t_pi_6_DME.gif")


MoviePy - Building file t_pi_6_DME.gif with imageio.
